In [190]:
import pandas as pd

df_bb = pd.read_csv('./data/imdb_barbie_Uncleaned.csv', header=None)
df_op = pd.read_csv('./data/imdb_oppenhimmer_Uncleaned.csv', header=None)

# Regular Expression

In [191]:
import re

score_Reg = '(\d{1,2})\/\d{1,2}\s.*\s'
title_Reg = '\d{1,2}\/\d{1,2}\s(\..*)'
people_found_helpful_Reg = '([\d,]*) out of ([\d,]*) found this helpful'
username_Reg = '\d{1,2}\/\d{1,2}\s.*\s([A-Za-z0-9_-]+)\d{2}\s[January|February|March|April|June|July|August|September|October|November|December]'
date_Reg = '(\d{1,2}\s\w+\s\d{4})'
review_Reg = '\d{1,2}\s\w+\s\d{4}\s([\s\S]*)\s[\d,]* out of [\d,]* found this helpful'

In [192]:
# Regular Expression test code

test_text = df_bb.iloc[18][0]

print(test_text)

res = re.findall(score_Reg, test_text)
print(res)

res = re.findall(title_Reg, test_text)
print(res)

res = re.findall(username_Reg, test_text)
print(res)

res = re.findall(date_Reg, test_text)
print(res)

res = re.findall(people_found_helpful_Reg, test_text)
print(res)

res = re.findall(review_Reg, test_text)
print(res)

Honestly - not funny and light enough
imdbmovieguy21 July 2023
I really enjoyed the first 20 minutes of the movie. It was very upbeat, positive and light. However, things soon became negative about 20 - 25 minutes into the movie. I couldn't stand it. Barbie's world and attitude was a constant bummer. Ken's attitude was sour. For the next hour (and more) the movie just lived in negativity.

I bought tickets to this show a month ago. And, I saw it the day before the official opening day. I was very excited about this movie. And, I was playing the music (Dua Lipa) song almost every day. I was really excited for this movie - and was looking forward to something very positive. But, the vast majority of the movie was the "conflict" part of the writing. It was just too much! The movie didn't need to go that way, that far, that fast. I would have preferred a more shallow and fun Barbie movie vs. This Barbie movie filled with over an hour of conflict.
358 out of 799 found this helpful. Was this

In [193]:
# Clean every row in data set

# cleand_df_bb = pd.DataFrame(columns=['score', 'title', 'username', 'date', 'people_found_helpful', 'people_didnt_find_helpful', 'review'])

def get_cleaned_df(dirty_df):
	cleaned_df = pd.DataFrame(columns=['score', 'title', 'username', 'date', 'people_found_helpful', 'review'])
    # cleaned_df = pd.DataFrame(columns=['score', 'title', 'username', 'date', 'people_found_helpful', 'people_didnt_find_helpful', 'review'])
	for index in range(len(dirty_df)):
		row = dirty_df.iloc[index][0]
		
		score = re.findall(score_Reg, row)
		title = re.findall(title_Reg, row)
		username = re.findall(username_Reg, row)
		date = re.findall(date_Reg, row)
		people_found_helpful = re.findall(people_found_helpful_Reg, row)
		review = re.findall(review_Reg, row)
		
		if not score:
			score = ['NULL']
		if not title:
			title = ['NULL']
		if not username:
			username = ['NULL']
		if not date:
			date = ['NULL']
		if not people_found_helpful:
			people_found_helpful = ['NULL', 'NULL']
		if not review:
			review = ['NULL']
			
		cleaned_df.loc[index] = [score[0], title[0], username[0], date[0], people_found_helpful[0], review[0]]
		
	return cleaned_df

In [194]:
cleand_df_bb = get_cleaned_df(df_bb)
cleand_df_bb.head()

,score,title,username,date,people_found_helpful,review
0,6,NULL,Imshortok06,23 July 2023,"(1,007, 1,363)",I'm just going to get straight to the point: I...
1,6,NULL,LoveofLegacy,21 July 2023,"(1,190, 1,646)","Margot does the best with what she's given, bu..."
2,6,NULL,Binkconn,21 July 2023,"(804, 1,173)","Barbie begins in an irresistible way, as a fun..."
3,8,NULL,coxaneesa,24 July 2023,"(152, 242)",Warning: Spoilers
4,9,NULL,heatherhilgers,24 July 2023,"(20, 26)","Wow, this movie was a love letter to cinema. F..."


In [ ]:
cleand_df_op = get_cleaned_df(df_op)
cleand_df_op.head()